In [ ]:
# https://www.kaggle.com/oscarsommervold/iceberg-challenge-exploration
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
data = pd.read_json('./data/train/data/processed/train.json')
test = pd.read_json('./data/test/data/processed/test.json')
data['band_1'] = data['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
data['band_2'] = data['band_2'].apply(lambda x: np.array(x).reshape(75, 75))
data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce')


# From the background section: " you will see data with two channels: HH (transmit/receive horizontally) and HV (transmit horizontally and receive vertically)."
# Composing a third band of horizontal and vertical feedback should then be as simple as band_3 = band_1 + band_2, since our data is in Decibel which is logarithmic.

band_1 = np.concatenate([im for im in data['band_1']]).reshape(-1, 75, 75)
band_2 = np.concatenate([im for im in data['band_2']]).reshape(-1, 75, 75)
band_3 = band_1 + band_2
full_img = np.stack([band_1, band_2, band_3], axis=1)
def createIm(d):
    imgs = []

    for i, row in d.iterrows():
    #make 75x75 image
        b_1 = np.array(row['band_1']).reshape(75, 75)
        b_2 = np.array(row['band_2']).reshape(75, 75)
        b_3 = b_1 + b_2
        
        r = (b_1 + abs(b_1.min())) / np.max((b_1 + abs(b_1.min())))
        g = (b_2 + abs(b_2.min())) / np.max((b_2 + abs(b_2.min()))) 
        b = (b_3 + abs(b_3.min())) / np.max((b_3 + abs(b_3.min())))

        imgs.append(np.dstack((r, g, b)))
    return np.array(imgs)

ship = data[data.is_iceberg == 0]
iceberg = data[data.is_iceberg == 1]
ship_img = createIm(ship)
iceberg_img = createIm(iceberg)

In [ ]:
import matplotlib.pyplot as plt
if not os.path.exists('./train/ship'):
    os.makedirs('train/ship')
if not os.path.exists('./train/iceberg'):
    os.makedirs('train/iceberg')
for i,x in enumerate(ship_img):
    plt.imsave('train/ship/ship_%d.png' % i,x)

for i,x in enumerate(iceberg_img):
    plt.imsave('train/iceberg/iceberg_%d.png' % i,x)

In [ ]:
test_img = createIm(test)
test_img.shape
os.makedirs('test/')
for i,x in enumerate(test_img):
    plt.imsave('test/test_%d.png' % i,x)

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np
# plt.style.use('science')

def plot_loss(n):
    # y = []
    enc = np.load('./result/alexnet-epoch30-32-adam-0.0001-pre-True/alexnet-epoch30-32-adam-0.0001-pre-True.npy')
    # for i in range(0,n):   
    #     tempy = list(enc)
    #     y += tempy
    x = range(0,n)
    y = list(enc)
    plt.plot(x, y, '.-')
    plt_title = 'BATCH_SIZE = 32; LEARNING_RATE:0.001'
    plt.title(plt_title)
    plt.xlabel('epoch')
    plt.ylabel('LOSS')
    # plt.savefig(file_name)
    plt.show()

plot_loss(30)

In [ ]:
False is True